In [1]:
import requests
import json
import pandas as pd
from datetime import date

# Get the latest feed
r = requests.get('https://api.nasa.gov/insight_weather/?api_key=DEMO_KEY&feedtype=json')

# Get the Status code from the API call
print("Response Status:")
print(r.status_code)

#Create the date for the Excel sheet name
today = date.today()
date_run = today.strftime("%m%d%Y")


# Normalize the json response and download it as an Excel sheet
df = pd.json_normalize(r.json())

# Use current timestamp if date_run is not defined
try:
    sheet_name = date_run
except NameError:
    sheet_name = datetime.now().strftime('%Y-%m-%d')

# Ensure sheet name is valid (Excel has restrictions on sheet names)
sheet_name = ''.join(c for c in sheet_name if c.isalnum() or c in [' ', '_', '-'])[:31]

# Export to Excel
df.to_excel('./latest_insight_data.xlsx', sheet_name=sheet_name, index=False)

# Get, convert, and print the response here
# Convert the response to text for later print out
insight_response = r.text

#Convert to JSON objects
json_object = json.loads(insight_response)

#Format to make it pretty
json_formatted_str = json.dumps(json_object, indent=2)

#Print the pretty JSON response
print("Response Content:")
print(json_formatted_str)

ModuleNotFoundError: No module named 'pandas'

In [3]:
import requests
import json
import pandas as pd
from datetime import date, datetime

def export_mars_insight_data_to_excel(response_json, date_run=None, output_file='latest_insight_data.xlsx'):
    """
    Export Mars Insight data to an Excel file with multiple sheets
    
    Args:
        response_json (dict): JSON response from Mars Insight API
        date_run (str, optional): Date to use in sheet name
        output_file (str): Path to output Excel file
    """
    # If response_json is a requests.Response object, get its JSON
    if hasattr(response_json, 'json'):
        response_json = response_json.json()
    
    # Safely extract data with default values
    def safe_get(data, *keys, default='N/A'):
        for key in keys:
            try:
                data = data[key]
            except (KeyError, TypeError):
                return default
        return data
    
    # Check if sol_keys exist
    sol_keys = response_json.get('sol_keys', [])
    
    # Use today's date if no date_run provided
    if date_run is None:
        date_run = date.today().strftime("%m%d%Y")
    
    # Ensure sheet name is valid (Excel has restrictions on sheet names)
    sheet_name = ''.join(c for c in date_run if c.isalnum() or c in [' ', '_', '-'])[:31]
    
    if not sol_keys:
        print("No sol data available in the response.")
        return None
    
    # Prepare Excel writer
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Summary Sheet
        summary_data = []
        
        for sol_key in sol_keys:
            # Validity Checks
            at_validity = safe_get(response_json, 'validity_checks', sol_key, 'AT', default={})
            hws_validity = safe_get(response_json, 'validity_checks', sol_key, 'HWS', default={})
            pre_validity = safe_get(response_json, 'validity_checks', sol_key, 'PRE', default={})
            
            summary_row = {
                'Sol': sol_key,
                'AT Hours with Data': ', '.join(map(str, safe_get(at_validity, 'sol_hours_with_data', default=[]))),
                'AT Valid': safe_get(at_validity, 'valid', default='Unknown'),
                'HWS Hours with Data': ', '.join(map(str, safe_get(hws_validity, 'sol_hours_with_data', default=[]))),
                'HWS Valid': safe_get(hws_validity, 'valid', default='Unknown'),
                'PRE Hours with Data': ', '.join(map(str, safe_get(pre_validity, 'sol_hours_with_data', default=[]))),
                'PRE Valid': safe_get(pre_validity, 'valid', default='Unknown')
            }
            summary_data.append(summary_row)
        
        # Create Summary DataFrame and write to Excel
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Optional: Add individual data sheets for each sol if needed
        for sol_key in sol_keys:
            try:
                sol_data = response_json[sol_key]
                sol_df = pd.json_normalize(sol_data)
                sol_df.to_excel(writer, sheet_name=f'Sol_{sol_key}', index=False)
            except Exception as e:
                print(f"Could not process data for Sol {sol_key}: {e}")
    
    print(f"Mars Insight data exported to {output_file}")
    return output_file

# Replace the existing Excel export code with this function call
export_mars_insight_data_to_excel(r)

Mars Insight data exported to latest_insight_data.xlsx


'latest_insight_data.xlsx'

In [1]:
import sys
import platform

print("Python Information:")
print("Executable:", sys.executable)
print("Version:", sys.version)
print("Platform:", platform.platform())

print("\nPython Path:")
for path in sys.path:
    print(path)

print("\nTrying to import libraries:")
import pandas as pd
print("Pandas version:", pd.__version__)

import numpy as np
print("Numpy version:", np.__version__)

Python Information:
Executable: /Users/coco/Documents/GitHub/getInsight/marsenv/bin/python
Version: 3.13.1 (main, Dec  3 2024, 17:59:52) [Clang 16.0.0 (clang-1600.0.26.4)]
Platform: macOS-15.1.1-arm64-arm-64bit-Mach-O

Python Path:
/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python313.zip
/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13
/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload

/Users/coco/Documents/GitHub/getInsight/marsenv/lib/python3.13/site-packages

Trying to import libraries:
Pandas version: 2.2.3
Numpy version: 2.2.2
